In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score, r2_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../data/lichess_db_standard_rated_2013-01.csv")
df.dtypes

Шахматные дебюты (они же ECO), кроются в самих ходах. В таком случае модели про них знать не обязательно (иначе зависимости в признаках). Это также касается опенингов

In [3]:
df['termination'].value_counts()

Конец игры может быть двух видов: закончилось время или игра  

In [4]:
df['results'].value_counts()

Результаты бывают 3ех видов: победа белых, победа черных и ничья

In [5]:
df['Events'].value_counts()

На самом деле поле time_control тоже зависит от типа события. Пока упростим задачу и будем просто угадывать для Blitz, Classical и Bullet партий рейтинги (турниры пока опустим)

In [6]:
df.head()

Итого хотим, зная последовательность ходов, тип игры, исходные рейтинги игроков угадывать рейтинги. Подготовим нашу дату.

У каждого хода есть точка старта и точка прихода. Давайте напишем функцию, которая по позиции на доске вернет уникальное число

Также напишем функцию, которая по точке старта и финиша (2 числа) вернет уникальное число конкретного хода

In [7]:
def get_cell_index(pos: str) -> int:  # return value in [0, 63]
    return 8 * (ord(pos[0]) - ord('a')) + (ord(pos[1]) - ord('1'))


def get_move_index(move: str) -> int:
    return 64 * get_cell_index(move[0] + move[1]) + get_cell_index(move[2] + move[3])


def change_all_moves(all_moves):
    res = []
    splt = all_moves.split("'")
    for i in range(1, len(splt), 2):
        res.append(get_move_index(splt[i]))
    return res

In [8]:
df['moves_indexes'] = df['all_moves'].apply(change_all_moves)

In [9]:
df.head()

Оставим только интересующие нас поля, пофильтруем их

In [10]:
df_clear = df[['Events', 'results', 'white_elo', 'black_elo', 'white_rating_diff', 'black_rating_diff', 'termination',
               'moves_indexes']]

In [11]:
df_clear.head()

In [12]:
df_clear = df_clear.loc[((df_clear['Events'] == 'Rated Classical game') | (
        df_clear['Events'] == 'Rated Bullet game') | (df_clear['Events'] == 'Rated Blitz game')) & (
                                abs(df_clear['white_rating_diff']) <= 10) & (
                                abs(df_clear['black_rating_diff']) <= 10)]

In [13]:
from rating_to_category import rating_to_number

df_test = df_clear[['Events', 'results', 'termination', 'moves_indexes']]
df_predict_white = df_clear['white_elo'].apply(rating_to_number)
df_predict_black = df_clear['black_elo'].apply(rating_to_number)

In [14]:
df_test.shape, df_predict_white.shape, df_predict_black.shape

In [15]:
encoded = pd.get_dummies(df_test, columns=['termination', 'Events', 'results'])

In [16]:
encoded.head()

Ходы мы не можем оставить как массив чисел. Давайте условимся, что в партии не может быть больше 300 ходов (в нашем датасете это число 360). Если ходов будет больше - просто обросим их. Напишем функцию, которая по ходам вернет новый датафрейм, который мы соеденим с нашим исходным

In [17]:
def get_new_df(moves):
    MAX_MOVE_COUNT = 300
    res = [[] for _ in range(MAX_MOVE_COUNT)]
    for my_moves in moves:
        for i in range(min(MAX_MOVE_COUNT, len(my_moves))):
            res[i].append(my_moves[i])
        for i in range(min(MAX_MOVE_COUNT, len(my_moves)), MAX_MOVE_COUNT):
            res[i].append(0)
    my_dict = {'move ' + str(i): res[i] for i in range(MAX_MOVE_COUNT)}
    return pd.DataFrame(my_dict)

In [18]:
res = get_new_df(encoded['moves_indexes'])
res.shape, encoded.shape

Теперь соединим наши датафреймы и уберем столбец moves_indexes (т.к. все ходы уже записаны в отдельных признаках)

In [19]:
total_df = pd.concat([encoded.reset_index(drop=True), res.reset_index(drop=True)], axis=1)
del total_df['moves_indexes']
total_df.head()

In [20]:
x_train_white, x_test_white, y_train_white, y_test_white = train_test_split(total_df, df_predict_white, train_size=0.8,
                                                                            random_state=42)
x_train_white.shape, x_test_white.shape, y_train_white.shape, y_test_white.shape

In [21]:
y_train_white.unique(), y_test_white.unique()

Обучим модель для предсказания только рейтинга белого. Рейтинг черного не должен сильно отличаться

In [22]:
models = {
    "LogisticRegression": LogisticRegression(),
    "RandomForestClassifier": RandomForestClassifier()
}
results = []
for name, model in models.items():
    model.fit(x_train_white, y_train_white)
    y_pred_test = model.predict(x_test_white)
    y_pred_train = model.predict(x_train_white)
    results.append({
        "model": name,
        "f1_score test": f1_score(y_true=y_test_white, y_pred=y_pred_test, average='micro'),
        "r2_score test": r2_score(y_true=y_test_white, y_pred=y_pred_test),
        "balanced_accuracy test": balanced_accuracy_score(y_true=y_test_white, y_pred=y_pred_test),
        "f1_score train": f1_score(y_true=y_train_white, y_pred=y_pred_train, average='micro'),
        "r2_score train": r2_score(y_true=y_train_white, y_pred=y_pred_train),
        "balanced_accuracy train": balanced_accuracy_score(y_true=y_train_white, y_pred=y_pred_train),
    })
total_res = pd.DataFrame(results)
total_res